# CH 8 - Data Wrangling: Join, Combine, and Reshape

## 8.1 Hierarchical Indexing

* Hierarchical indexing is an important feature of pandas that enables you to have mul‐
tiple (two or more) index levels on an axis. Somewhat abstractly, it provides a way for
you to work with higher dimensional data in a lower dimensional form. Let’s start
with a simple example; create a Series with a list of lists (or arrays) as the index:

In [1]:
import numpy as np
import pandas as pd

data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    1.963840
   2    0.753049
   3   -1.239779
b  1   -0.124691
   3   -2.248644
c  1    0.593084
   2    0.234193
d  2    1.100473
   3   -0.804009
dtype: float64

In [3]:
# partial indexing - select subsets of data
data['b']

1    0.754068
3   -0.169164
dtype: float64

In [4]:
data['b':'c']

b  1    0.754068
   3   -0.169164
c  1   -1.597141
   2   -1.270670
dtype: float64

In [5]:
data.loc[['b', 'd']]

b  1    0.754068
   3   -0.169164
d  2    0.477612
   3   -0.104260
dtype: float64

In [6]:
data.loc[:, 2]

a   -0.426682
c   -1.270670
d    0.477612
dtype: float64

In [7]:
#  you could rearrange the data into 
# a DataFrame using its unstack method:

data.unstack()

,1,2,3
a,0.170209,-0.426682,0.353363
b,0.754068,NaN,-0.169164
c,-1.597141,-1.270670,NaN
d,NaN,0.477612,-0.104260


In [8]:
# Reverse

data.unstack().stack()

a  1    0.170209
   2   -0.426682
   3    0.353363
b  1    0.754068
   3   -0.169164
c  1   -1.597141
   2   -1.270670
d  2    0.477612
   3   -0.104260
dtype: float64

In [9]:
# With a DataFrame, either axis can have a hierarchical index:

frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [10]:
# The hierarchical levels can have names (as strings or any Python objects). 
# If so, these will show up in the console output:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']

frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [11]:
# With partial column indexing you can similarly select groups of columns:

frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [12]:
# A MultiIndex can be created by itself and then reused; 
# the columns in the preceding DataFrame with level names
# could be created like this:

pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], 
                           ['Green', 'Red', 'Green']],
                          names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Reordering and Sorting Levels

In [13]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [17]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [18]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics by Level

In [21]:
frame.sum(level='key2') # by rows

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [22]:
frame.sum(level='color', axis=1) # by cols

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame’s columns

In [25]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                      'd':  [0, 1, 2, 0, 1, 2, 3]})

frame

In [28]:
frame2 = frame.set_index(['c', 'd'])

frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [30]:
# By default the columns are removed from the DataFrame, 
# though you can leave them in:

frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [31]:
# reset_index does the opposite

frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 Combining and Merging Datasets

### Merging on Index

* In some cases, the merge key(s) in a DataFrame will be found in its index. In this
case, you can pass left_index=True or right_index=True (or both) to indicate that
the index should be used as the merge key

In [8]:
 left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                       'value': range(6)})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [9]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
right1

,group_val
a,3.5
b,7.0


In [10]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


#### Hierarchically indexed Merge

* With hierarchically indexed data, things are more complicated, as joining on index is
implicitly a multiple-key merge:

In [11]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})

lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [12]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [13]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [14]:
# Using the indexes of both sides of the merge is also possible:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [15]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [16]:
# merge method
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [17]:
# join method
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


* DataFrame has a convenient join instance for merging by index. It can also be used
to combine together many DataFrame objects having the same or similar indexes but
non-overlapping columns. 
*  DataFrame’s join
method performs a left join on the join keys, exactly preserving the left frame’s row
index. It also supports joining the index of the passed DataFrame on one of the columns of the calling DataFrame:

In [18]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


* Lastly, for simple index-on-index merges, you can pass a list of DataFrames to join as
an alternative to using the more general concat function described in the next
section:

In [19]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])

another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [20]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


### Concatenating Along an Axis

* Another kind of data combination operation is referred to interchangeably as concat‐
enation, binding, or stacking. NumPy’s concatenate function can do this with
NumPy arrays:

In [21]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [22]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In the context of pandas objects such as Series and DataFrame, having labeled axes
enable you to further generalize array concatenation. In particular, you have a number of additional things to think about:
* If the objects are indexed differently on the other axes, should we combine the
distinct elements in these axes or use only the shared values (the intersection)?
* Do the concatenated chunks of data need to be identifiable in the resulting
object?
* Does the “concatenation axis” contain data that needs to be preserved? In many
cases, the default integer labels in a DataFrame are best discarded during
concatenation.

In [2]:
#  Suppose we have three Series with no index overlap:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'b', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [24]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
b    3
e    4
f    5
g    6
dtype: int64

In [25]:
# By default concat works along axis=0, producing another Series. If you pass axis=1,
# the result will instead be a DataFrame (axis=1 is the columns)
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,3.0,NaN
c,NaN,2.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


* In this case there is no overlap on the other axis, which as you can see is the sorted
union (the 'outer' join) of the indexes. You can instead intersect them by passing
join='inner':

In [3]:
s4 = pd.concat([s1,s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [27]:
pd.concat([s1, s4])

a    0
b    1
a    0
b    1
f    5
g    6
dtype: int64

In [28]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [29]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [33]:
# You can even specify the axes to be used on the other axes with reindex (join_axes deprecated):
pd.concat([s1, s4], axis=1).reindex(['a', 'c', 'b', 'e'])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


* A potential issue is that the concatenated pieces are not identifiable in the result. Suppose instead you wanted to create a hierarchical index on the concatenation axis. To
do this, use the keys argument

In [4]:
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    c    2
       b    3
       e    4
three  f    5
       g    6
dtype: int64

In [5]:
result.unstack()

,a,b,c,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN
two,NaN,3.0,2.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,5.0,6.0


In [7]:
# In the case of combining Series along axis=1, 
# the keys become the DataFrame column headers:

pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,3.0,NaN
c,NaN,2.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [51]:
# The same logic extends to DataFrame objects

df1 = pd.DataFrame(np.arange(6).reshape(3, 2), 
                   index=['a', 'b', 'c'],
                   columns=['one', 'two'])

df1

,one,two
a,0,1
b,2,3
c,4,5


In [52]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), 
                   index=['a', 'c'],
                   columns=['three', 'four'])

df2

,three,four
a,5,6
c,7,8


In [22]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [25]:
# If you pass a dict of objects instead of a list, 
# the dict’s keys will be used for the keys option:

pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [53]:
# we can name the created axis levels with the names argument

pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [42]:
# A last consideration concerns DataFrames in which the 
# row index does not contain any relevant data

df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df1.index = [4, 5, 6]
df1

,a,b,c,d
4,-0.065221,-0.629732,-0.317208,-0.064367
5,-0.999994,-0.851849,-0.082720,-0.548089
6,0.090015,-1.593581,-0.365577,-0.480227


In [43]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df2

,b,d,a
0,0.605475,-0.803633,0.939169
1,-0.262234,-1.517878,-1.192156


In [44]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-0.065221,-0.629732,-0.317208,-0.064367
1,-0.999994,-0.851849,-0.082720,-0.548089
2,0.090015,-1.593581,-0.365577,-0.480227
3,0.939169,0.605475,NaN,-0.803633
4,-1.192156,-0.262234,NaN,-1.517878


### Combining Data with Overlap

In [58]:
# As a motivating example, consider NumPy’s where function, which performs
# the array-oriented equivalent of an if-else expression:

a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [59]:
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [60]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, 5. ])

In [62]:
# Series has a combine_first method, which performs the equivalent of 
# this operation along with pandas’s usual data alignment logic

b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

* With DataFrames, combine_first does the same thing column by column, so you
can think of it as “patching” missing data in the calling object with data from the
object you pass:

In [71]:
#     Signature: pd.DataFrame.combine_first(self, other: 'DataFrame') -> 'DataFrame'
#     Docstring:
#     Update null elements with value in the same location in `other`.

#     Combine two DataFrame objects by filling null values in one DataFrame
#     with non-null values from other DataFrame. The row and column indexes
#     of the resulting DataFrame will be the union of the two.

In [65]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [66]:
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [68]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 8.3 Reshaping and Pivoting

### Reshaping with Hierarchical Indexing
Hierarchical indexing provides a consistent way to rearrange data in a DataFrame.
There are two primary actions:

* `stack`
   * This “rotates” or pivots from the columns in the data to the rows 
* `unstack`
    * This pivots from the rows into the columns

In [3]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [4]:
# Using the stack method on this data pivots the columns 
# into the rows, producing a Series:

result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [5]:
# From a hierarchically indexed Series, you can rearrange the data 
# back into a DataFrame with unstack:

result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [6]:
# By default the innermost level is unstacked (same with stack). 
# You can unstack a different level by passing a level number or name:

result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [7]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


#### Unstacking might introduce missing data if all of the values in the level aren’t found in each of the subgroups

In [8]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [9]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


#### Stacking filters out missing data by default, so the operation is more easily invertible

In [10]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [11]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

#### When you unstack in a DataFrame, the level unstacked becomes the lowest level in the result

In [12]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [13]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

### Pivoting “Long” to “Wide” Format

In [15]:
data = pd.read_csv('examples/macrodata.csv')

In [16]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [19]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [20]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [21]:
data = data.reindex(columns=columns)

In [22]:
data.head()

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2


In [26]:
data.index = periods.to_timestamp('D', 'end')

In [25]:
?pd.PeriodIndex.to_timestamp

Signature: pd.PeriodIndex.to_timestamp(self, *args, **kwargs)
Docstring:
Cast to DatetimeArray/Index.

Parameters
----------
freq : str or DateOffset, optional
    Target frequency. The default is 'D' for week or longer,
    'S' otherwise.
how : {'s', 'e', 'start', 'end'}
    Whether to use the start or end of the time period being converted.

Returns
-------
DatetimeArray/Index
File:      ~/.local/miniconda3/envs/jupyter/lib/python3.8/site-packages/pandas/core/accessor.py
Type:      function


In [27]:
data.head()

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2


In [31]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})

ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


####  You might prefer to have a DataFrame containing one column per distinct item value indexed by timestamps 
####  in the date column. DataFrame’s pivot method performs exactly this transformation

In [32]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


#### The first two values passed are the columns to be used respectively as the row and column index, then finally an optional value column to fill the DataFrame. 
####  Suppose you had two value columns that you wanted to reshape simultaneously:

In [35]:
ldata['value2'] = np.random.randn(len(ldata))

ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-1.575200
1,1959-03-31 23:59:59.999999999,infl,0.000,0.292930
2,1959-03-31 23:59:59.999999999,unemp,5.800,1.013062
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,0.894338
4,1959-06-30 23:59:59.999999999,infl,2.340,0.980683
5,1959-06-30 23:59:59.999999999,unemp,5.100,0.386119
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,-0.888947
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.785114
8,1959-09-30 23:59:59.999999999,unemp,5.300,-1.270841
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,0.546375


#### By omitting the last argument, you obtain a DataFrame with hierarchical columns:

In [37]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.292930 -1.575200   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  0.980683  0.894338   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.785114 -0.888947   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.988722  0.546375   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.177140 -1.902679   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.013062  
1959-06-30 23:59:59.999999999  0.386119  
1959-09-30 23:59:59.999999999 -1.270841  
1959-12-31 23:59:59.999999999  0.591112  
1960-03-31 23:59:59.999999999  1.256617

#### Note that pivot is equivalent to creating a hierarchical index using set_index followed by a call to unstack:

In [39]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')

unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.292930 -1.575200   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  0.980683  0.894338   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.785114 -0.888947   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.988722  0.546375   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.177140 -1.902679   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -1.006148 -2.234141   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6 -0.184220 -1.372906   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.013062  
1959-06-30 23:59:59.999999999  0.386119  
1959-09-30 23:59:59.999999999 -1.270841  
1959-12-31 23:59:59.999999999  0.591112  
1960-03-31 23:59:59.999999999  1.256617  
1960-06-30 23:59:59.999999999  1.385595  
1960-09-30 23:59:59.999999999 -0.250094

### Pivoting “Wide” to “Long” Format

#### An inverse operation to pivot for DataFrames is pandas.melt. 

In [41]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


#### The 'key' column may be a group indicator, and the other columns are data values.
* When using pandas.melt, we must indicate which columns (if any) are group indicators. Let’s use 'key' as the only group indicator here:

In [42]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


#### Using pivot, we can reshape back to the original layout:

In [43]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


#### Since the result of pivot creates an index from the column used as the row labels, 
* we may want to use reset_index to move the data back into a column

In [44]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


#### You can also specify a subset of columns to use as value columns:

In [45]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


#### pandas.melt can be used without any group identifiers, too:

In [46]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [47]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
